- Carga el archivo iris.csv.
- Codifica la columna class como variable numérica (LabelEncoder).
- Normaliza los datos con MinMaxScaler.
- Separa en entrenamiento (80%) y prueba (20%) con semilla 42.

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler 
from sklearn.model_selection import train_test_split

df = pd.read_csv("iris.csv", sep=",")
X = df.drop("class", axis=1)
y = LabelEncoder().fit_transform(df["class"])

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
print("Datos cargados y preparados")

Entrena los siguientes modelos y compara su exactitud:

- Regresión logística
- Perceptrón
- SVM lineal
- KNN (k=5)

In [ ]:
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

modelos ={
    "Regresion logistica": LogisticRegression(max_iter=1000),
    "Perceptron": Perceptron(max_iter=1000),
    "SVM lineal": SVC(kernel="linear"),
    "KNN (k=5)": KNeighborsClassifier(n_neighbors=5)
}

for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    acc = accuracy_score(y_test, modelo.predict(X_test))
    print(f"{modelo}: {acc:.2f}")

- Usa sepal length y petal width para entrenar una SVM lineal.
- Dibuja la superficie de decisión y los vectores soporte. ¿Cuántos vectores soporte hay?
- Repite con C=0.1, C=1 y C=10.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv("iris.csv")
df = df[df["class"].isin(["Iris-setosa", "Iris-versicolor"])]  # Binario

# Seleccionar dos variables
X = df[["sepal length", "petal width"]]
y = LabelEncoder().fit_transform(df["class"])  # 0 y 1

# Normalizar y dividir
X_scaled = MinMaxScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Función para graficar para cada valor de C
def graficar_svm_lineal(C_val):
    clf = SVC(kernel='linear', C=C_val)
    clf.fit(X_train, y_train)

    # Dibujar muestras
    for clase in np.unique(y_train):
        plt.scatter(X_train[y_train == clase, 0], X_train[y_train == clase, 1], label=f"Clase {clase}")

    # Vectores soporte
    plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], c='k', marker='*', label='Vectores soporte')

    # Recta de decisión
    m = -clf.coef_[0, 0] / clf.coef_[0, 1]
    n = -clf.intercept_[0] / clf.coef_[0, 1]
    x_vals = np.linspace(X_train[:, 0].min(), X_train[:, 0].max(), 100)
    plt.plot(x_vals, m * x_vals + n, 'k-', label='Recta de decisión')

    plt.xlabel("sepal length")
    plt.ylabel("petal width")
    plt.title(f"SVM Lineal (C={C_val})")
    plt.legend()
    plt.grid(True)
    plt.show()

    print(f"C={C_val} -> Vectores soporte: {len(clf.support_)}")

# Ejecutar para los valores deseados de C
for C in [0.1, 1, 10]:
    graficar_svm_lineal(C)



Ejercicio 3: Multiclase con KNN
- Prueba con k = 1, k = 3, k = 9.

In [ ]:
for k in [1, 3, 9]:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"Knn (k={k}) Accuracy: {acc:.4f}")


Vas a desarrollar una aplicación de visión por computador cuyo objetivo es determinar el género de una persona al entrar en un local. Para ello, trabajarás con la base de datos CelebA, la cual incluye, para cada imagen, el género de la persona y un conjunto de características visuales como formato, color, forma facial, entre otras.

Estas características no se extraen directamente de las imágenes, sino que han sido preprocesadas previamente por otro equipo, utilizando un algoritmo basado en redes neuronales para generar representaciones numéricas de cada imagen.

Tu tarea se centra en la parte de clasificación, es decir, en diseñar, entrenar y evaluar un modelo que, a partir de estas características extraídas, sea capaz de predecir correctamente el género de las personas.

1.Leer el conjunto de entrenamiento y de test de los archivos CelebA-1K-train.csv
y CelebA-1K-test.csv respectivamente.

In [ ]:
train_df = pd.read_csv("CelebA-1K-train.csv")
test_df = pd.read_csv("CelebA-1K-test.csv")

X_train_vals = train_df.drop(columns="Gender")
y_train_vals = train_df["Gender"]
X_test_vals = test_df.drop(columns="Gender")
y_test_vals = test_df["Gender"]

2.Entrenar un clasificador lineal biclásico utilizando el método de Regresión Logística
y otro usando el perceptron obtener la tasa de acierto de los clasificadorres en las muestras del conjunto de test utilizar la función accuracy_score

In [ ]:
# Regresión Logística
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_vals, y_train_vals)  # Se usa y_train_vals, no y_test_vals
y_pred_log = log_reg.predict(X_test_vals)
acc_log = accuracy_score(y_test_vals, y_pred_log)

# Perceptrón
percep = Perceptron(max_iter=1000)
percep.fit(X_train_vals, y_train_vals)
y_pred_percep = percep.predict(X_test_vals)
acc_percep = accuracy_score(y_test_vals, y_pred_percep)  # Nombre distinto para no sobrescribir

3.Clasificar las imágenes que están en el archivo comprimido ImagenesParaClasificar.zip
y cuyas características están en el conjunto de datos de test CelebA-1K-test.csv,
indicando cuales de ellas las clasifica incorrectamente el clasificador

In [ ]:
import os

def clasificar_imagenes(clf, df_test, carpeta_imagenes='ImagenesParaClasificar'):
    """
    Clasifica las imágenes en una carpeta utilizando un clasificador ya entrenado
    y un DataFrame con la información correspondiente.

    Parámetros:
    - clf: clasificador entrenado (por ejemplo, un modelo de sklearn)
    - df_test: DataFrame con columnas ['Image_name', 'Genero', ...características]
    - carpeta_imagenes: nombre de la carpeta con las imágenes a clasificar
    """
    nombres = os.listdir(carpeta_imagenes)
    print(f'Se encontraron {len(nombres)} imágenes para clasificar.\n')

    for nombre in nombres:
        indice = df_test['Image_name'] == nombre
        fila = df_test[indice]
        
        if fila.empty:
            print(f'{nombre}: no se encontró en el DataFrame.\n')
            continue

        genero = fila.values[0, 1]
        X = fila.values[:, 2:]

        genero_pred = clf.predict(X)
        print(f'{nombre} → género etiquetado: {genero}')
        print(f'{nombre} → género predicho:  {genero_pred[0]}\n')



1. Leer el dataset de precios de casas, normaliza y dividelo en train y test. (80% - 20%)

In [ ]:
df = pd.read_csv("PrecioCasas.csv", sep = ",")
X = df.drop(columns="Precio")
y = df["Precio"]

scaled = MinMaxScaler()
X_scaled = scaled.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

2. Medir el tiempo de entrenamiento y test para los métodos `ball_tree`, `kd_tree` y `brute` del KNN usando el conjunto completo de características con jobs = 1.
    - ¿Hay diferencias significativas en el tiempo de ejecución entre los métodos?

In [ ]:
import time

# Métodos a evaluar
algorithms = ['ball_tree', 'kd_tree', 'brute']
resultados = {}

for algo in algorithms:
    knn = KNeighborsClassifier(algorithm=algo, n_neighbors=5, n_jobs=1)

    # Medir tiempo de entrenamiento
    t0 = time.time()
    knn.fit(X_train, y_train)
    t_entrenamiento = time.time() - t0

    # Medir tiempo de predicción
    t1 = time.time()
    y_pred = knn.predict(X_test)
    t_test = time.time() - t1

    acc = accuracy_score(y_test, y_pred)
    
    resultados[algo] = {
        'tiempo_entrenamiento': t_entrenamiento,
        'tiempo_test': t_test,
        'accuracy': acc
    }

resultados_df = pd.DataFrame(resultados).T
print(resultados_df)


### **Ejercicio: Impacto de la Normalización y del Parámetro de Regularización en SVM**

En este ejercicio, se analizará cómo afectan la normalización de las variables de entrada y el parámetro de regularización `C` al rendimiento de un clasificador SVM.

Utilizando los conjuntos de datos `precio_casas_clasificacion.csv` y `CelebA-2K.csv` con 80% train y 20% test, realiza las siguientes tareas:

1. **Entrenamiento sin normalización:**
   - Entrena un **clasificador lineal** y un **SVM con diferentes kernels** (por ejemplo: lineal, RBF, polinómico) usando las variables **sin normalizar**.
   - Observa y comenta los resultados. Ten en cuenta que algunas variables, como en el caso del dataset de precios de casas, pueden tener rangos muy distintos.

2. **Entrenamiento con normalización:**
   - Aplica **dos técnicas de normalización** a las variables de entrada:
     - **Escalado lineal (MinMaxScaler)**
     - **Estandarización (StandardScaler)**
   - Repite el entrenamiento con los mismos modelos (lineal y SVM con diferentes kernels) y compara los resultados con los obtenidos en el paso anterior.

3. **Análisis del parámetro `C`:**
   - Para cada configuración anterior (con y sin normalización), prueba diferentes valores del **parámetro de regularización `C`** (por ejemplo: `0.01`, `0.1`, `1`, `10`, `100`).
   - Analiza cómo afecta este parámetro a la **exactitud del modelo** en el conjunto de prueba.

4. **Compara con otros clasificadores el rendimento:**
   - KNN
   - Regresión logística
   - Perceptrón

4. **Conclusión:**
   - Resume tus observaciones sobre cómo influyen la normalización y el parámetro `C` en el rendimiento de los modelos SVM.
   - Comenta si hay diferencias significativas entre los dos datasets en cuanto a sensibilidad a la normalización.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Perceptron

# Cargar datasets
df_casas = pd.read_csv("precio_casas_clasificacion.csv")
X = df_casas.drop(columns="Precio")
y = df_casas["Precio"]
scaled = MinMaxScaler()
X_scaled = scaled.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"Logistic Regression: {acc*100:.4f}")

svm = SVC(kernel="linear")
svm.fit(X_train, y_train)
y_pred1 = svm.predict(X_test)
acc1 = accuracy_score(y_test, y_pred1)
print(f"SVM: {acc1*100:.4f}")



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Perceptron

# Cargar datasets
df_casas = pd.read_csv("precio_casas_clasificacion.csv")
X = df_casas.drop(columns="Precio")
y = df_casas["Precio"]
scaled = StandardScaler()
X_scaled = scaled.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"Logistic Regression: {acc*100:.4f}")

svm = SVC(kernel="linear")
svm.fit(X_train, y_train)
y_pred1 = svm.predict(X_test)
acc1 = accuracy_score(y_test, y_pred1)
print(f"SVM: {acc1*100:.4f}")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Perceptron

# Cargar datasets
df_casas = pd.read_csv("precio_casas_clasificacion.csv")
X = df_casas.drop(columns="Precio")
y = df_casas["Precio"]
scaled = MinMaxScaler()
X_scaled = scaled.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"Logistic Regression: {acc*100:.4f}%")

svm = SVC(kernel="linear")
svm.fit(X_train, y_train)
y_pred1 = svm.predict(X_test)
acc1 = accuracy_score(y_test, y_pred1)
print(f"SVM: {acc1*100:.4f}%")

percep = Perceptron(max_iter=1000)
percep.fit(X_train, y_train)
y_pred2 = percep.predict(X_test)
acc2 = accuracy_score(y_test, y_pred2)
print(f"Perceptron: {acc2*100:.4f}%")

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred3 = knn.predict(X_test)
acc3 = accuracy_score(y_test, y_pred3)
print(f"KNN: {acc3*100:.4f}%")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Perceptron

# Cargar datasets
df_celeb = pd.read_csv("CelebA-2K.csv")
X = df_celeb.drop(columns=["Image_name", "Gender"])
y = df_celeb["Gender"]
scaled = StandardScaler()
X_scaled = scaled.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"Logistic Regression: {acc*100:.4f}%")

svm = SVC(kernel="linear")
svm.fit(X_train, y_train)
y_pred1 = svm.predict(X_test)
acc1 = accuracy_score(y_test, y_pred1)
print(f"SVM: {acc1*100:.4f}%")

La normalizacion te ayuda a que todas las colunnas tengan un valor equitativo, ya que puede que hayan columnas con valores muy grande por lo que el modelo podria darle más peso a esta, aunque no sea la más importante. En la regresión logistica, en este caso, si no se normaliza, el modelo no congerje, además ayuda a que el alogritmo llegue mas rapido y mejor a la solución optima

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Perceptron

# Cargar datasets
df_celeb = pd.read_csv("CelebA-2K.csv")
X = df_celeb.drop(columns=["Image_name", "Gender"])
y = df_celeb["Gender"]
scaled = MinMaxScaler()
X_scaled = scaled.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"Logistic Regression: {acc*100:.4f}%")

svm = SVC(kernel="linear")
svm.fit(X_train, y_train)
y_pred1 = svm.predict(X_test)
acc1 = accuracy_score(y_test, y_pred1)
print(f"SVM: {acc1*100:.4f}%")

percep = Perceptron(max_iter=1000)
percep.fit(X_train, y_train)
y_pred2 = percep.predict(X_test)
acc2 = accuracy_score(y_test, y_pred2)
print(f"Perceptron: {acc2*100:.4f}%")

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred3 = knn.predict(X_test)
acc3 = accuracy_score(y_test, y_pred)
print(f"KNN: {acc3*100:.4f}%")

La normalizacion te ayuda a que todas las colunnas tengan un valor equitativo, ya que puede que hayan columnas con valores muy grande por lo que el modelo podria darle más peso a esta, aunque no sea la más importante. En la regresión logistica, en este caso, si no se normaliza, el modelo no congerje, además ayuda a que el alogritmo llegue mas rapido y mejor a la solución optima. El archivo "CelebA-2k.csv" es mejor para cualquier modelo.

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Perceptron

df = pd.read_csv("iris.csv", sep = ",")
X = df.drop(columns="class")
y = df["class"]

escaler = MinMaxScaler()
X_scaled = escaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"Regresion logica: {acc*100:.4f}")

percep = Perceptron()
percep.fit(X_train, y_train)
y_pred = percep.predict(X_test)
acc2 = accuracy_score(y_test, y_pred)

print(f"Perceptron: {acc2*100:.4f}")

svm = SVC(kernel="linear")
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"SVM: {acc*100:.4f}")

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"KNN: {acc*100:.4f}")

Regresion logica: 96.6667
Perceptron: 83.3333
SVM: 100.0000
KNN: 100.0000
